# 最適化問題（シフトスケジューリング問題）をPuLPで解く際のサンプルコード

## 参考: 
- [組合せ最適化でナーススケジューリング問題を解く](https://qiita.com/SaitoTsutomu/items/a33aba1a95828eb6bd3f)
- [「組合せ最適化でナーススケジューリング問題を解く」を自分なりに理解してみた](https://naomichi.work/2018/11/21/2018-11-21-175004/)

### メモ:
- 実はインターフェースは**PuLP**じゃなくて**Python-MIP**の方が同梱されている商用利用できるリゾルバが頭良くてこのほうがいいのでは？との意見が散見される。

### Step1: モジュールインポート

In [1]:
import numpy as np, pandas as pd
from pulp import *
from ortoolpy import addvars, addbinvars
from io import StringIO

### Step2: データ読み込みと整形

### 変数, 定数の意味対応一覧

- shift_h: 希望シフトのデータフレーム
- time_zone: 朝、昼, 夜などの時間帯
- require: 必要人数
- staff_N: スタッフN
- dow: 曜日（Day of weekの略)
- FRAME_TOTAL: コマの総数
- STAFF_TOTAL: 従業員総数
- STAFF_IDX_LIST: 従業員のインデックスリスト
- ADMIN_STAFF_IDX_LIST: 管理者権限従業員のインデックスリスト
- PW_N: 制約条件Nのペナルティweight
    - DIFF_REQUIRE: 必要人数差
    - CANNOT_ASSIGN: 希望不可シフトへのアサイン
    - MIN_ASSING: 最低コマ数（各従業員に対して、希望シフトの1/2）
    - LACK_ADMIN: 管理者不足
- assign_frame: 従業員数×コマ数の0-1変数
- is_require_diff: 必要人数差がある（必要人数がアサイン人数を下回る）かどうかのフラグ
- is_lack_admin: 管理者不足かどうかのフラグ
- min_assign: 最低コマ数を満たしているかどうかのフラグを書くのした配列


In [2]:
# 希望シフトのデータ
shift_h = pd.read_table(StringIO("""\
曜日\t月\t月\t月\t火\t火\t火\t水\t水\t水\t木\t木\t木\t金\t金\t金\t土\t土\t土\t日\t日\t日
時間帯\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜\t朝\t昼\t夜
必要人数\t2\t3\t3\t2\t3\t3\t2\t3\t3\t1\t2\t2\t2\t3\t3\t2\t4\t4\t2\t4\t4
従業員0\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t
従業員1\t○\t○\t○\t\t\t\t○\t○\t○\t\t\t\t○\t○\t○\t\t\t\t\t\t
従業員2\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○
従業員3\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○
従業員4\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○
従業員5\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t\t\t\t\t\t
従業員6\t\t\t\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○\t○\t○\t○
従業員7\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t
従業員8\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○\t\t\t○
従業員9\t\t\t\t\t\t\t\t\t\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○\t○""")).T

In [3]:
# 0行目を削除してカラムを英語化してDataframeにつける
shift_h.columns = ['time_zone', 'require', 'staff_0', 'staff_1', 'staff_2', 'staff_3', 'staff_4', 'staff_5', 'staff_6', 'staff_7', 'staff_8', 'staff_9']
shift_h = shift_h.iloc[1:]

# 必要人数をint型に変換
shift_h.require = shift_h.require.astype(int)

# 希望シフトの有無をbooleanにビット変換
shift_h.iloc[:,2:] = ~shift_h.iloc[:,2:].isnull()

# 曜日を入れ込み1文字目を適用
shift_h.insert(0, 'dow', shift_h.index.str[0])

# インデックスリセット
shift_h.reset_index(drop=True, inplace=True)

# dow, require, time_zoneをカラムの後ろに移動
shift_h = shift_h.iloc[:,list(range(3,shift_h.shape[1]))+[0,1,2]]

In [4]:
# DFが意図通り作られているか確認
display(shift_h[:3])

,staff_0,staff_1,staff_2,staff_3,staff_4,staff_5,staff_6,staff_7,staff_8,staff_9,dow,time_zone,require
0,True,True,False,True,False,True,False,False,False,False,月,朝,2
1,False,True,False,True,False,True,False,True,False,False,月,昼,3
2,False,True,False,True,True,True,False,False,True,False,月,夜,3


### Step3: モデルに用いる定数と変数を定義

In [5]:
# 必要な定数化
FRAME_TOTAL = shift_h.shape[0]
STAFF_TOTAL = shift_h.shape[1]-3
STAFF_IDX_LIST = list(range(STAFF_TOTAL))
ADMIN_STAFF_IDX_LIST = [3, 5, 9]

# 各種ペナルティweight    
PW_DIFF_REQUIRE = 10
PW_CANNOT_ASSIGN = 100
PW_MIN_ASSIGN = 10
PW_LACK_ADMIN = 100


# 従業員数×コマ数の0-1変数を格納した2次元配列の作成。これがアサイン用のシフト表になる。
assign_frame =  np.array(addbinvars(FRAME_TOTAL, STAFF_TOTAL))

# 必要人数差を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで差があるかないかのフラグとして利用)
shift_h['is_require_diff'] = addvars(FRAME_TOTAL)

#  最低コマ数を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで最低アサインを満たしているかどうかのフラグとして利用)
min_assign = addvars(STAFF_TOTAL)

#  管理者不足を保存するための0-1変数を格納した1次元配列を作成し、DFのカラムに追加
# (あくまで管理者不足かどうかのフラグとして利用)
shift_h['is_lack_admin'] = addvars(FRAME_TOTAL)

### Step4: 定式化

In [6]:
# 数理モデル作成(最小化問題)
model = LpProblem()

# 目的関数定義
# 目的関数1: 必要人数が一致していない場合
function_1 = PW_DIFF_REQUIRE * lpSum(shift_h.is_require_diff)

# 目的関数2: 希望不可にアサインしてしまった時。制約も巻き込んでる?
# rがそのシフト時間帯の行
# shift_h.apply(r: 1-r[STAFF_IDX_LIST])がシフトrのスタッフNが希望しているかどうかをのフラグをまとめたもの、希望してない部分が1になる（1 - False = 1, 1 -True = 0）。
# assign_frame[r.name]がシフトrのアサイン情報
# shift_h.apply(lambda r: lpDot(1-r[STAFF_IDX_LIST], assign_frame[r.name]), 1)が希望してないところにシフトが割り当てられてしまっている部分の一覧
function_2 = PW_CANNOT_ASSIGN * lpSum(shift_h.apply(lambda r: lpDot(1-r[STAFF_IDX_LIST], assign_frame[r.name]), 1))

# 目的関数3: 最低コマ数を満たしていない時
function_3 = PW_MIN_ASSIGN * lpSum(min_assign)

# 目的関数4: 管理者不足が存在する時
function_4 = PW_LACK_ADMIN * lpSum(shift_h.is_lack_admin)

# 最終的な目的関数
model += function_1 + function_2 + function_3 + function_4

# 制約付与
# 制約1: 必要人数が一致していない場合制約をつける。プラマイそれぞれについて
# 制約2: 管理者がいない場合に制約つける
for _, r in shift_h.iterrows():
    model += r.is_require_diff >=  (lpSum(assign_frame[r.name]) - r.require)
    model += r.is_require_diff >= -(lpSum(assign_frame[r.name]) - r.require)
    model += lpSum(assign_frame[r.name, ADMIN_STAFF_IDX_LIST]) + r.is_lack_admin >= 1

# 制約3: 希望シフトが1/2以上アサインされていない場合制約を付与する
for j ,n in enumerate((shift_h.iloc[:,STAFF_IDX_LIST].sum(0)+1)//2):
    model += lpSum(assign_frame[:, j]) + min_assign[j] >= n

### Step5: 実行

In [7]:
# 実行
%time model.solve()

CPU times: user 43 ms, sys: 11 ms, total: 54 ms
Wall time: 188 ms


1

### Step6: 結果表示

In [8]:
# 結果表示

result = np.vectorize(value)(assign_frame).astype(int)
shift_h['result'] = [''.join(i*j for i, j in zip(r, shift_h.columns)) for r in result]
print('目的関数: ', value(model.objective))
print('---------')
print('結果')
display(shift_h[['dow','time_zone','result']])

目的関数:  0.0
---------
結果


,dow,time_zone,result
0,月,朝,staff_1staff_5
1,月,昼,staff_1staff_3staff_7
2,月,夜,staff_1staff_3staff_4
3,火,朝,staff_0staff_3
4,火,昼,staff_3staff_5staff_7
5,火,夜,staff_4staff_5staff_8
6,水,朝,staff_0staff_5
7,水,昼,staff_1staff_5staff_7
8,水,夜,staff_3staff_4staff_8
9,木,朝,staff_3
